# Data Processing & Transformation
## Brazilian E-commerce Dataset - Phase 2

This notebook processes the raw Olist data and prepares it for:
- Recommendation engine development
- Multi-region deployment simulation
- MinIO object storage integration

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pickle
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set paths
data_dir = Path('../data')
raw_dir = data_dir / 'raw'
processed_dir = data_dir / 'processed'
processed_dir.mkdir(exist_ok=True)

print(f"✅ Processing data from: {raw_dir}")
print(f"✅ Saving processed data to: {processed_dir}")

✅ Processing data from: ..\data\raw
✅ Saving processed data to: ..\data\processed


## 1. Load and Clean Core Datasets

In [2]:
# Load main datasets
orders = pd.read_csv(raw_dir / 'olist_orders_dataset.csv')
customers = pd.read_csv(raw_dir / 'olist_customers_dataset.csv')
order_items = pd.read_csv(raw_dir / 'olist_order_items_dataset.csv')
products = pd.read_csv(raw_dir / 'olist_products_dataset.csv')
reviews = pd.read_csv(raw_dir / 'olist_order_reviews_dataset.csv')
sellers = pd.read_csv(raw_dir / 'olist_sellers_dataset.csv')
geolocation = pd.read_csv(raw_dir / 'olist_geolocation_dataset.csv')

print("Dataset shapes:")
print(f"Orders: {orders.shape}")
print(f"Customers: {customers.shape}")
print(f"Order Items: {order_items.shape}")
print(f"Products: {products.shape}")
print(f"Reviews: {reviews.shape}")
print(f"Sellers: {sellers.shape}")
print(f"Geolocation: {geolocation.shape}")

Dataset shapes:
Orders: (99441, 8)
Customers: (99441, 5)
Order Items: (112650, 7)
Products: (32951, 9)
Reviews: (99224, 7)
Sellers: (3095, 4)
Geolocation: (1000163, 5)


## 2. Data Quality Assessment & Cleaning

In [3]:
# Check data quality
def assess_data_quality(df, name):
    print(f"\n=== {name} ===")
    print(f"Shape: {df.shape}")
    print(f"Missing values: {df.isnull().sum().sum()}")
    print(f"Duplicate rows: {df.duplicated().sum()}")
    
    if df.isnull().sum().sum() > 0:
        print("Columns with missing values:")
        null_cols = df.isnull().sum()[df.isnull().sum() > 0]
        for col, count in null_cols.items():
            print(f"  {col}: {count} ({count/len(df)*100:.1f}%)")

# Assess all datasets
assess_data_quality(orders, "Orders")
assess_data_quality(customers, "Customers")
assess_data_quality(order_items, "Order Items")
assess_data_quality(products, "Products")
assess_data_quality(reviews, "Reviews")
assess_data_quality(sellers, "Sellers")


=== Orders ===
Shape: (99441, 8)
Missing values: 4908
Duplicate rows: 0
Columns with missing values:
  order_approved_at: 160 (0.2%)
  order_delivered_carrier_date: 1783 (1.8%)
  order_delivered_customer_date: 2965 (3.0%)

=== Customers ===
Shape: (99441, 5)
Missing values: 0
Duplicate rows: 0

=== Order Items ===
Shape: (112650, 7)
Missing values: 0
Columns with missing values:
  order_approved_at: 160 (0.2%)
  order_delivered_carrier_date: 1783 (1.8%)
  order_delivered_customer_date: 2965 (3.0%)

=== Customers ===
Shape: (99441, 5)
Missing values: 0
Duplicate rows: 0

=== Order Items ===
Shape: (112650, 7)
Missing values: 0
Duplicate rows: 0

=== Products ===
Shape: (32951, 9)
Missing values: 2448
Duplicate rows: 0
Columns with missing values:
  product_category_name: 610 (1.9%)
  product_name_lenght: 610 (1.9%)
  product_description_lenght: 610 (1.9%)
  product_photos_qty: 610 (1.9%)
  product_weight_g: 2 (0.0%)
  product_length_cm: 2 (0.0%)
  product_height_cm: 2 (0.0%)
  product_

## 3. Create Master Dataset for Recommendations

In [4]:
# Create comprehensive dataset by joining all tables
print("Creating master dataset...")

# Start with orders and add customer info
master_df = orders.merge(customers, on='customer_id', how='left')
print(f"After adding customers: {master_df.shape}")

# Add order items
master_df = master_df.merge(order_items, on='order_id', how='left')
print(f"After adding order items: {master_df.shape}")

# Add product info
master_df = master_df.merge(products, on='product_id', how='left')
print(f"After adding products: {master_df.shape}")

# Add seller info
master_df = master_df.merge(sellers, on='seller_id', how='left')
print(f"After adding sellers: {master_df.shape}")

# Add reviews (optional - not all orders have reviews)
master_df = master_df.merge(reviews[['order_id', 'review_score', 'review_creation_date']], 
                           on='order_id', how='left')
print(f"After adding reviews: {master_df.shape}")

print(f"\n✅ Master dataset created: {master_df.shape}")
print(f"Columns: {list(master_df.columns)}")

Creating master dataset...
After adding customers: (99441, 12)
After adding order items: (113425, 18)
After adding products: (113425, 26)
After adding order items: (113425, 18)
After adding products: (113425, 26)
After adding sellers: (113425, 29)
After adding reviews: (114092, 31)

✅ Master dataset created: (114092, 31)
Columns: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm', 'seller_zip_code_prefix', 'seller_city', 'seller_state', 'review_score', 'review_creation_date']
After adding sellers

## 4. Feature Engineering for Recommendations

In [5]:
# Clean and engineer features
print("Engineering features for recommendation system...")

# Convert dates
date_columns = ['order_purchase_timestamp', 'order_approved_at', 
                'order_delivered_carrier_date', 'order_delivered_customer_date', 
                'order_estimated_delivery_date', 'review_creation_date']

for col in date_columns:
    if col in master_df.columns:
        master_df[col] = pd.to_datetime(master_df[col], errors='coerce')

# Create implicit ratings based on multiple factors
def create_implicit_rating(row):
    """
    Create implicit rating (1-5) based on:
    - Review score (if available)
    - Order status
    - Price tier
    """
    # Start with review score if available
    if pd.notna(row['review_score']):
        return row['review_score']
    
    # Fall back to order status
    if row['order_status'] == 'delivered':
        return 4.0  # Assume satisfied if delivered
    elif row['order_status'] in ['shipped', 'processing']:
        return 3.5
    elif row['order_status'] == 'canceled':
        return 1.0
    else:
        return 3.0  # Neutral for unknown status

master_df['implicit_rating'] = master_df.apply(create_implicit_rating, axis=1)

# Create purchase value tiers
master_df['price_tier'] = pd.cut(master_df['price'], 
                                bins=5, 
                                labels=['budget', 'low', 'mid', 'high', 'premium'])

# Extract date features
master_df['purchase_year'] = master_df['order_purchase_timestamp'].dt.year
master_df['purchase_month'] = master_df['order_purchase_timestamp'].dt.month
master_df['purchase_weekday'] = master_df['order_purchase_timestamp'].dt.dayofweek

print(f"✅ Features engineered. Rating distribution:")
print(master_df['implicit_rating'].value_counts().sort_index())

Engineering features for recommendation system...
✅ Features engineered. Rating distribution:
implicit_rating
1.0    14795
2.0     3936
3.0     9498
3.5       92
4.0    22175
5.0    63596
Name: count, dtype: int64
✅ Features engineered. Rating distribution:
implicit_rating
1.0    14795
2.0     3936
3.0     9498
3.5       92
4.0    22175
5.0    63596
Name: count, dtype: int64


## 5. Regional Data Splits for Multi-Region Simulation

In [6]:
# Analyze Brazilian regions for multi-region simulation
print("Analyzing geographic distribution...")

# Map Brazilian states to regions
region_mapping = {
    # Southeast (Primary region - São Paulo)
    'SP': 'southeast', 'RJ': 'southeast', 'MG': 'southeast', 'ES': 'southeast',
    
    # South (Secondary region)
    'PR': 'south', 'SC': 'south', 'RS': 'south',
    
    # Northeast
    'BA': 'northeast', 'PE': 'northeast', 'CE': 'northeast', 'PB': 'northeast',
    'RN': 'northeast', 'AL': 'northeast', 'SE': 'northeast', 'PI': 'northeast', 'MA': 'northeast',
    
    # North
    'AM': 'north', 'PA': 'north', 'AC': 'north', 'RO': 'north', 
    'RR': 'north', 'AP': 'north', 'TO': 'north',
    
    # Central-West
    'GO': 'central_west', 'MT': 'central_west', 'MS': 'central_west', 'DF': 'central_west'
}

master_df['customer_region'] = master_df['customer_state'].map(region_mapping)
master_df['seller_region'] = master_df['seller_state'].map(region_mapping)

# Show regional distribution
print("\nCustomer distribution by region:")
region_dist = master_df['customer_region'].value_counts()
print(region_dist)
print(f"\nTotal orders: {len(master_df)}")

# For multi-region simulation, we'll use:
# - Southeast + South as "Region 1" (US-East simulation)
# - Northeast + North + Central-West as "Region 2" (EU-West simulation)

master_df['deployment_region'] = master_df['customer_region'].map({
    'southeast': 'region_1',
    'south': 'region_1', 
    'northeast': 'region_2',
    'north': 'region_2',
    'central_west': 'region_2'
})

print("\nDeployment region distribution:")
print(master_df['deployment_region'].value_counts())
print(f"Region 1: {master_df['deployment_region'].value_counts()['region_1']} orders ({master_df['deployment_region'].value_counts()['region_1']/len(master_df)*100:.1f}%)")
print(f"Region 2: {master_df['deployment_region'].value_counts()['region_2']} orders ({master_df['deployment_region'].value_counts()['region_2']/len(master_df)*100:.1f}%)")

Analyzing geographic distribution...

Customer distribution by region:
customer_region
southeast       78429
south           16348
northeast       10517
central_west     6713
north            2085
Name: count, dtype: int64

Total orders: 114092

Deployment region distribution:
deployment_region
region_1    94777
region_2    19315
Name: count, dtype: int64
Region 1: 94777 orders (83.1%)
Region 2: 19315 orders (16.9%)


## 6. Create Recommendation Datasets

In [7]:
# Create user-item matrix for collaborative filtering
print("Creating recommendation datasets...")

# Filter for completed orders only
completed_orders = master_df[master_df['order_status'].isin(['delivered', 'shipped'])].copy()
print(f"Completed orders: {len(completed_orders)}")

# Create user-item rating matrix
user_item_matrix = completed_orders.pivot_table(
    index='customer_unique_id',
    columns='product_category_name', 
    values='implicit_rating',
    aggfunc='mean',
    fill_value=0
)

print(f"\nUser-item matrix shape: {user_item_matrix.shape}")
print(f"Users (customers): {user_item_matrix.shape[0]}")
print(f"Items (categories): {user_item_matrix.shape[1]}")

# Calculate sparsity
sparsity = (user_item_matrix == 0).sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1])
print(f"Matrix sparsity: {sparsity:.3f} ({sparsity*100:.1f}% empty)")

# Create product popularity scores
product_popularity = completed_orders.groupby('product_category_name').agg({
    'order_id': 'count',
    'implicit_rating': 'mean',
    'price': 'mean'
}).rename(columns={
    'order_id': 'purchase_count',
    'implicit_rating': 'avg_rating',
    'price': 'avg_price'
})

product_popularity['popularity_score'] = (
    product_popularity['purchase_count'] * 0.5 + 
    product_popularity['avg_rating'] * 0.5
)

print(f"\nTop 10 most popular product categories:")
print(product_popularity.sort_values('popularity_score', ascending=False).head(10))

Creating recommendation datasets...
Completed orders: 112037
Completed orders: 112037

User-item matrix shape: (93115, 73)
Users (customers): 93115
Items (categories): 73
Matrix sparsity: 0.986 (98.6% empty)

Top 10 most popular product categories:
                        purchase_count  avg_rating   avg_price  \
product_category_name                                            
cama_mesa_banho                  11227    3.904471   93.356733   
beleza_saude                      9638    4.163675  129.757057   
esporte_lazer                     8593    4.140929  113.119564   
moveis_decoracao                  8325    3.933514   87.151422   
informatica_acessorios            7791    3.962328  116.064527   
utilidades_domesticas             6898    4.087054   90.763400   
relogios_presentes                5940    4.044697  200.356013   
telefonia                         4493    3.974182   69.749664   
ferramentas_jardim                4321    4.063643  110.323136   
automotivo               

## 7. Save Processed Datasets

In [8]:
# Save processed datasets
print("Saving processed datasets...")

# Main datasets
master_df.to_csv(processed_dir / 'master_dataset.csv', index=False)
completed_orders.to_csv(processed_dir / 'completed_orders.csv', index=False)

# Recommendation matrices
user_item_matrix.to_csv(processed_dir / 'user_item_matrix.csv')
product_popularity.to_csv(processed_dir / 'product_popularity.csv')

# Regional splits
region_1_data = completed_orders[completed_orders['deployment_region'] == 'region_1']
region_2_data = completed_orders[completed_orders['deployment_region'] == 'region_2']

region_1_data.to_csv(processed_dir / 'region_1_orders.csv', index=False)
region_2_data.to_csv(processed_dir / 'region_2_orders.csv', index=False)

# Create summary statistics
summary_stats = {
    'processing_date': datetime.now().isoformat(),
    'total_orders': len(master_df),
    'completed_orders': len(completed_orders),
    'unique_customers': master_df['customer_unique_id'].nunique(),
    'unique_products': master_df['product_id'].nunique(),
    'product_categories': master_df['product_category_name'].nunique(),
    'date_range': {
        'start': master_df['order_purchase_timestamp'].min().isoformat() if pd.notna(master_df['order_purchase_timestamp'].min()) else None,
        'end': master_df['order_purchase_timestamp'].max().isoformat() if pd.notna(master_df['order_purchase_timestamp'].max()) else None
    },
    'regions': {
        'region_1': len(region_1_data),
        'region_2': len(region_2_data)
    },
    'user_item_matrix': {
        'shape': user_item_matrix.shape,
        'sparsity': float(sparsity)
    }
}

import json
with open(processed_dir / 'processing_summary.json', 'w') as f:
    json.dump(summary_stats, f, indent=2, default=str)

print("\n✅ All datasets saved to processed/ directory:")
for file in processed_dir.glob('*'):
    print(f"  - {file.name}")

print(f"\n📊 Processing Summary:")
print(f"  Total orders: {summary_stats['total_orders']:,}")
print(f"  Completed orders: {summary_stats['completed_orders']:,}")
print(f"  Unique customers: {summary_stats['unique_customers']:,}")
print(f"  Product categories: {summary_stats['product_categories']}")
print(f"  Region 1 orders: {summary_stats['regions']['region_1']:,}")
print(f"  Region 2 orders: {summary_stats['regions']['region_2']:,}")
print(f"  User-item matrix: {summary_stats['user_item_matrix']['shape']}")
print(f"  Matrix sparsity: {summary_stats['user_item_matrix']['sparsity']:.1%}")

Saving processed datasets...

✅ All datasets saved to processed/ directory:
  - completed_orders.csv
  - master_dataset.csv
  - processing_summary.json
  - product_popularity.csv
  - region_1_orders.csv
  - region_2_orders.csv
  - user_item_matrix.csv

📊 Processing Summary:
  Total orders: 114,092
  Completed orders: 112,037
  Unique customers: 96,096
  Product categories: 73
  Region 1 orders: 92,997
  Region 2 orders: 19,040
  User-item matrix: (93115, 73)
  Matrix sparsity: 98.6%

✅ All datasets saved to processed/ directory:
  - completed_orders.csv
  - master_dataset.csv
  - processing_summary.json
  - product_popularity.csv
  - region_1_orders.csv
  - region_2_orders.csv
  - user_item_matrix.csv

📊 Processing Summary:
  Total orders: 114,092
  Completed orders: 112,037
  Unique customers: 96,096
  Product categories: 73
  Region 1 orders: 92,997
  Region 2 orders: 19,040
  User-item matrix: (93115, 73)
  Matrix sparsity: 98.6%


## 8. Data Quality Validation

In [9]:
# Validate processed data
print("Validating processed datasets...")

def validate_dataset(df, name, required_columns=None):
    print(f"\n=== Validation: {name} ===")
    print(f"✅ Shape: {df.shape}")
    
    # Check for required columns
    if required_columns:
        missing_cols = set(required_columns) - set(df.columns)
        if missing_cols:
            print(f"❌ Missing columns: {missing_cols}")
        else:
            print(f"✅ All required columns present")
    
    # Check for duplicates
    duplicates = df.duplicated().sum()
    print(f"📊 Duplicate rows: {duplicates}")
    
    # Check data types
    print(f"📊 Data types: {df.dtypes.value_counts().to_dict()}")
    
    return df.shape[0] > 0 and (not required_columns or not missing_cols)

# Validate key datasets
validations = {
    'Master Dataset': validate_dataset(
        master_df, 
        'Master Dataset',
        ['customer_unique_id', 'product_id', 'implicit_rating', 'deployment_region']
    ),
    'Completed Orders': validate_dataset(
        completed_orders,
        'Completed Orders', 
        ['customer_unique_id', 'product_category_name', 'implicit_rating']
    ),
    'User-Item Matrix': validate_dataset(
        user_item_matrix,
        'User-Item Matrix'
    )
}

all_valid = all(validations.values())
print(f"\n{'✅' if all_valid else '❌'} Overall validation: {'PASSED' if all_valid else 'FAILED'}")

if all_valid:
    print("\n🎉 Data processing completed successfully!")
    print("📁 Processed files ready for MinIO upload and database loading")
    print("⏭️  Next step: Run scripts/setup_minio.py")
else:
    print("\n⚠️  Some validations failed. Please review the data processing steps.")

Validating processed datasets...

=== Validation: Master Dataset ===
✅ Shape: (114092, 39)
✅ All required columns present
📊 Duplicate rows: 141
📊 Data types: {dtype('O'): 15, dtype('float64'): 13, dtype('<M8[ns]'): 6, dtype('int32'): 3, dtype('int64'): 1, CategoricalDtype(categories=['budget', 'low', 'mid', 'high', 'premium'], ordered=True, categories_dtype=object): 1}

=== Validation: Completed Orders ===
✅ Shape: (112037, 39)
✅ All required columns present
📊 Duplicate rows: 141
📊 Data types: {dtype('O'): 15, dtype('float64'): 13, dtype('<M8[ns]'): 6, dtype('int32'): 3, dtype('int64'): 1, CategoricalDtype(categories=['budget', 'low', 'mid', 'high', 'premium'], ordered=True, categories_dtype=object): 1}

=== Validation: Completed Orders ===
✅ Shape: (112037, 39)
✅ All required columns present
📊 Duplicate rows: 140
📊 Data types: {dtype('O'): 15, dtype('float64'): 13, dtype('<M8[ns]'): 6, dtype('int32'): 3, dtype('int64'): 1, CategoricalDtype(categories=['budget', 'low', 'mid', 'high', '